In [1]:
import requests as rq
import pandas as pd
import os
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
from dotenv import load_dotenv

load_dotenv()

coins = ["bitcoin", "ethereum"]
all_data = []
df_coins = pd.DataFrame()

for coin in coins:
    urlCompany = f"{os.environ.get('VARIABLE-COMPANIES-URL')}/{coin}"
    responseCompany = rq.get(urlCompany)

    if responseCompany.status_code == 200:
        data = responseCompany.json()
        df_company = pd.json_normalize(data["companies"])
        df_company["id"] = coin
        all_data.append(df_company)
    else:
        print(f"Error with {coin}: {responseCompany.status_code}")

df_companies = pd.concat(all_data, ignore_index=True)

urlCoin = os.environ.get("VARIABLE-COINS-URL")
responseCoin = rq.get(urlCoin)

if responseCoin.status_code == 200:
    df_coins = pd.json_normalize(responseCoin.json())
else:
    print(f"Error: {responseCoin.status_code}")

dataframe = pd.merge(df_companies, df_coins, on="id")
display(dataframe.head(200))


,name_x,symbol_x,country,total_holdings,total_entry_value_usd,total_current_value_usd,percentage_of_total_supply,id,symbol_y,name_y,...,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,roi.times,roi.currency,roi.percentage
0,Strategy,NASDAQ:MSTR,US,629376,46076000000,70074127549,2.997,bitcoin,btc,Bitcoin,...,-10.35687,2025-08-14T00:37:02.582Z,67.810000,1.639966e+05,2013-07-06T00:00:00.000Z,NaN,2025-08-27T02:21:33.101Z,NaN,NaN,NaN
1,MARA Holdings,NASDAQ:MARA,US,50639,0,5638098283,0.241,bitcoin,btc,Bitcoin,...,-10.35687,2025-08-14T00:37:02.582Z,67.810000,1.639966e+05,2013-07-06T00:00:00.000Z,NaN,2025-08-27T02:21:33.101Z,NaN,NaN,NaN
2,Twenty One Capital (XXI),NASDAQ:CEP,US,43514,0,4844807533,0.207,bitcoin,btc,Bitcoin,...,-10.35687,2025-08-14T00:37:02.582Z,67.810000,1.639966e+05,2013-07-06T00:00:00.000Z,NaN,2025-08-27T02:21:33.101Z,NaN,NaN,NaN
3,Bullish,NYSE:BLSH,US,24000,0,2672137261,0.114,bitcoin,btc,Bitcoin,...,-10.35687,2025-08-14T00:37:02.582Z,67.810000,1.639966e+05,2013-07-06T00:00:00.000Z,NaN,2025-08-27T02:21:33.101Z,NaN,NaN,NaN
4,"Riot Platforms, Inc",NASDAQ:RIOT,US,19239,0,2142052032,0.092,bitcoin,btc,Bitcoin,...,-10.35687,2025-08-14T00:37:02.582Z,67.810000,1.639966e+05,2013-07-06T00:00:00.000Z,NaN,2025-08-27T02:21:33.101Z,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,BTC Digital,NASDAQ: BTCT,US,2135,5000000,9760050,0.002,ethereum,eth,Ethereum,...,-7.77664,2025-08-24T19:21:03.333Z,0.432979,1.053395e+06,2015-10-20T00:00:00.000Z,NaN,2025-08-27T02:21:33.654Z,53.844083,btc,5384.408316
90,Mogo Inc.,NASDAQ:MOGO,CA,146,405880,667432,0.000,ethereum,eth,Ethereum,...,-7.77664,2025-08-24T19:21:03.333Z,0.432979,1.053395e+06,2015-10-20T00:00:00.000Z,NaN,2025-08-27T02:21:33.654Z,53.844083,btc,5384.408316
91,DigitalX,ASX:DCC,AU,129,0,589717,0.000,ethereum,eth,Ethereum,...,-7.77664,2025-08-24T19:21:03.333Z,0.432979,1.053395e+06,2015-10-20T00:00:00.000Z,NaN,2025-08-27T02:21:33.654Z,53.844083,btc,5384.408316
92,Neptune Digital Assets Corp.,TSXV: DASH,CA,37,0,169143,0.000,ethereum,eth,Ethereum,...,-7.77664,2025-08-24T19:21:03.333Z,0.432979,1.053395e+06,2015-10-20T00:00:00.000Z,NaN,2025-08-27T02:21:33.654Z,53.844083,btc,5384.408316


In [2]:
dataframe.to_csv("file.csv", index=False)

SCOPES = ['https://www.googleapis.com/auth/drive.file']

creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(os.environ.get("VARIABLE-AUTH-SECRET"), SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

drive_service = build('drive', 'v3', credentials=creds)

file_metadata = {
    'name': 'file.csv',
    'parents': [os.environ.get("VARIABLE-FOLDER-ID")]
}
media = MediaFileUpload("file.csv", mimetype='text/csv')

file = drive_service.files().create(
    body=file_metadata,
    media_body=media,
    fields='id'
).execute()

print("File uploaded to GDrive successfully")

File uploaded to GDrive successfully
